In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with pd.ExcelFile(r"D:\小鸡理财\百度云同步盘\小鸡理财\数据报告\用户类报告\流失用户分析\流失用户分布.xlsx") as xlsx:
    df_jd_ds = pd.read_excel(xlsx,'经典待收变化')
    df_cg_ds = pd.read_excel(xlsx,'存管待收变化')
    df_vip = pd.read_excel(xlsx,'VIP变动')
    df_tz = pd.read_excel(xlsx,'投资')
    df_members = pd.read_excel(xlsx,'会员列表')

In [3]:
df_new_member = pd.read_csv(r'D:\小鸡理财\百度云同步盘\小鸡理财\数据报告\用户类报告\流失用户分析\运营数据.txt',
                            delimiter="\t",
                            header=None,
                            engine = 'python',
                            names=['用户名','手机号码','身份证'])

确定 经典版待收变化 表的用户对应的存管用户名

In [4]:
df_jd_ds.dropna(subset=['身份证'],inplace=True)

In [5]:
df_jd_ds['身份证'] = df_jd_ds['身份证'].str.upper()

In [7]:
df_new_member['身份证'] = df_new_member['身份证'].str.upper()

In [8]:
df_temp = pd.merge(df_jd_ds,df_new_member,
                   on = '身份证',
                   how = 'left',
                   suffixes=('_经典', '_存管'))

In [9]:
df_jd_ds_new = df_temp.loc[df_temp['用户名_存管'].notnull(),:]

In [50]:
df_cg_ds.columns

Index(['用户名', '真实姓名', '交易类型', '待收金额', '发生时间', '注册时间', '身份证', '用户来源'], dtype='object')

In [46]:
df_jd_ds_new = df_jd_ds_new.loc[:,['真实姓名','待收金额', '发生时间', '注册时间', '身份证', '用户来源','用户名_存管']]

合并 经典、存管待收变化 表

In [51]:
df_jd_ds_new.rename(columns={'用户名_存管':'用户名'},inplace=True)

In [55]:
df_jd_ds_new.columns

Index(['真实姓名', '待收金额', '发生时间', '注册时间', '身份证', '用户来源', '用户名'], dtype='object')

In [57]:
df_ds = pd.concat([df_jd_ds_new,df_cg_ds],axis = 0,join='inner')

In [58]:
df_ds.head()

,真实姓名,待收金额,发生时间,注册时间,身份证,用户来源,用户名
0,王秀珍,41800.00,2018-09-18 09:50:02,2018-01-18 19:04:56,370112198511230029,非存管版,XJLC_83710256077e830f
1,徐孟珍,10450.00,2018-09-18 09:50:02,2018-01-17 09:51:01,33021919620422022X,非存管版,XJLC_70236255704e902e
2,冯海英,41850.08,2018-09-18 09:50:02,2018-01-09 08:26:39,440106197312094024,非存管版,XJLC_7497125467999005
3,赵世杰,12540.00,2018-09-18 09:50:02,2018-01-06 00:07:28,420624197510072930,非存管版,XJLC_3875625430301af5
4,王琼,5226.72,2018-09-18 09:50:02,2018-01-05 11:48:33,330226196304120123,非存管版,XJLC_383062312384ce47


In [59]:
df_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159758 entries, 0 to 79254
Data columns (total 7 columns):
真实姓名    159758 non-null object
待收金额    159758 non-null float64
发生时间    159758 non-null datetime64[ns]
注册时间    159758 non-null object
身份证     159758 non-null object
用户来源    159758 non-null object
用户名     159758 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 9.8+ MB


筛选 待收表 中待收0的用户

In [61]:
df_ds_0 = df_ds[df_ds['待收金额'] == 0]

In [62]:
df_ds_0.shape

(6018, 7)

In [66]:
df_ds_0 = df_ds_0.copy()
df_ds_0.sort_values(by=['发生时间','用户名'],ascending=True,inplace=True)

In [69]:
df_ds_0.head(30)

,真实姓名,待收金额,发生时间,注册时间,身份证,用户来源,用户名
48536,姚露露,0.0,2018-03-05 15:05:01,2014-09-03 08:45:07,330902199201252925,非存管版,XJLC_28242404ccb3
48535,李雪燕,0.0,2018-03-05 15:05:01,2014-09-03 08:47:22,330227199210311500,非存管版,XJLC_30239417c94e
48540,黄振兴,0.0,2018-03-05 15:05:01,2014-09-02 20:53:14,620121198702155330,非存管版,XJLC_6576910e6010
48526,项晶晶,0.0,2018-03-05 15:05:02,2014-09-03 10:16:08,342422198710110869,非存管版,XJLC_1151066cac53
48490,黄玉莹,0.0,2018-03-05 15:05:02,2014-12-14 23:35:08,452701197010210047,非存管版,XJLC_179733714ead16
48529,吴丹婷,0.0,2018-03-05 15:05:02,2014-09-03 09:14:34,331022198812130225,非存管版,XJLC_2019853d7822
48454,陈蕾芬,0.0,2018-03-05 15:05:02,2015-04-16 11:33:23,330227198912185364,非存管版,XJLC_224254564f9250
48458,王林,0.0,2018-03-05 15:05:02,2015-03-30 12:12:42,230206198202240511,非存管版,XJLC_2382944901a386
48514,徐建华,0.0,2018-03-05 15:05:02,2014-09-24 13:39:23,362330197609095792,非存管版,XJLC_241754235ff4a
48475,林珍金,0.0,2018-03-05 15:05:02,2015-01-19 12:39:48,350321197305137711,非存管版,XJLC_245514233acf7d


查看 在待收变0 后是否还有投资，记录投资金额、投资时间

In [70]:
df_tz.head()

,用户名,真实姓名,手机号码,身份证号码,注册时间,计划期限,总投资金额,投资时间,投资来源
0,XJLC_1436714351992cb4,吕丽莎,18020211598,130427198707203926,2017-03-02,3月,5000.0,2018-06-30 23:42:11,安卓端
1,XJLC_36793269782f668e,任碧华,15258336620,330283199606075226,2018-06-07,1月,500.0,2018-06-30 23:33:06,安卓端
2,XJLC_36793269782f668e,任碧华,15258336620,330283199606075226,2018-06-07,3月,2000.0,2018-06-30 23:28:12,安卓端
3,XJLC_29720262583681a7,金政怡,13957536175,330602198804100022,2018-03-21,1月,36000.0,2018-06-30 23:07:42,安卓端
4,XJLC_222471780481373f,杨安芬,13777238953,330205197709060627,2017-06-20,1月,5000.0,2018-06-30 23:04:53,安卓端
